<img src='otus.png'>

In [3]:
import gensim
from gensim import corpora
import json
import os
from pprint import pprint
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

import pandas as pd
from nltk import FreqDist



# Анализ текстовых данных


Николенко, Кадурин, Архангельская. **Глубокое обучение. Погружение в мир нейронных сетей**. Глава 7.


### Какие задачи можно решать, обрабатывая текст?
"Мама мыла раму, и теперь она блестит"  
"Мама мыла раму, и теперь она сильно устала"  

"Кубок не помещался в чемодан, потому что он был слишком велик. Что именно было слишком велико, чемодан или кубок?"

http://commonsensereasoning.org/winograd.html


1. синтаксические задачи
  * разметка по частям речи и по морфологическим признакам
  * деление слов в тексте на морфемы (суффикс, приставка и пр.)
  * стемминг, лемматизация (?)
  * деление на предложения (инициалы и сокращения) и слова (китайский язык)
  * поиск имен и названий в тексте - сущностей
  * разрешение смысла слов в заданном контексте (замок)
  * построить синтаксическое дерево
  * определение того, к каким другим объектам относится слово
2. задачи на понимание текста, в которых есть "учитель"
  * предсказание следующего символа
  * информационный поиск
  * анализ тональности
  * выделение отношений и фактов
  * ответы на вопросы
3. понимание и порождение текста (оценка качества?)
  * порождение текста
  * автоматическое реферирование
  * машинный перевод
  * диалоговые модели (чат-бот)
  
Косвенные задачи:
  * описание изображения
  * распознавание речи
  
**Задачи бизнеса**:
  * распознавание речи (помощник)
  * чат-бот (замена техподдержки в решении большинства вопросов)
  * поиск точного ответа на вопрос в базе документов (например, база стандартов)
  * оценка мнения в социальных сетях о продукте
  * ... (ваши варианты?)

# Тематическое моделирование

Тематическая модель автоматически определяет, к каким темам относится каждый документ из коллекции документов, а так же какие слова (термины) характеризуют каждую тему.

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d5/%D0%A2%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.png">

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 154.402s.


In [2]:
data_samples[:2]

[u"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n",
 u"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little le

In [3]:


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.550s.
Extracting tf features for LDA...
done in 0.475s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.367s.

Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope

In [4]:
lda.components_


array([[  4.96604155,   4.3537397 ,  21.42539886, ...,   1.57926488,
          1.33933502,   1.20988436],
       [  0.48391034,   1.85845783,  14.04720958, ...,  74.59501615,
         59.36116266,   0.27698642],
       [  0.18708486,   0.13728929,   0.31409364, ...,   1.02679042,
          2.56259123,   0.13662652],
       ..., 
       [  3.22343848,  39.1368944 ,  11.24910558, ...,  23.37779481,
          3.06315114,   0.15230766],
       [  1.41871388,  47.53082031,  16.14390001, ...,  82.46751192,
         16.51319941,  28.11660323],
       [  4.02759659,   1.24781464,  13.26101699, ...,  29.0225105 ,
          0.24834416,   0.13033208]])

In [5]:
lda.components_.shape

(10, 1000)

# Наивный Байес
* Знаем метку каждого документа
* У каждого документа только одна метка  


Что можно сделать, если нет информации о метках?
#### Проблема кластеризации  
Можно решать с помощью EM-алгоритма:
* E-шаг - вычислить ожидания того, какой документ к какой теме относится
* M-шаг - с помощью Наивного Байеса определить вероятности $p(w|t)$ при фиксированных метках


## EM-алгоритм

Решает задачу кластеризации.  
Подбирает некоторые параметры модели для данных в которых неизвестен ответ.  

Expectation шаг:
* зафиксировать параметры модели
* посчитать значения скрытых переменных
Maximization шаг:
* зафиксировать скрытые переменные
* посчитать параметры модели

Повторять до сходимости.

Есть математическое обоснование того, что метод сходится к локальному экстремуму, на каждом шаге значение функции правдоподобия не убывает (правдоподобие $p(\theta | \mathcal{X})$ - насколько правдоподобна модель при данных параметрах, насколдько она хорошо описывает данные)

Частный случай EM-алгоритма - **k-means**.  
Метки кластеров - скрытые переменные Z  (latent variables)  
Параметры модели - центры кластеров  

<img src="kmeans.png">

Еще вариант EM-алгоритма - разделение смеси гауссиан (Gaussian Mixture Model, GMM)

Параметры модели - центр кластера и матрица ковариаций (здесь описывает форму могомерного нормального распраделения, или гауссианы)
Скрытые переменные - вероятность пренадлежности к каждой гауссиане (метка кластера выбирается как наиболее вероятный кластер)


<img src="gauss.png">

## PLSA

Что если у каждого документа может быть много меток?

Рассмотрим модель:
* Каждое слово в документе $d$ сгенерировано из некоторой темы $t \in T$
* Документ сгенерирован некоторым распределением над темами $p(t|d)$
* Слово сгенерировано из темы (не из документа) $p(w|d, t) = p(w|d)$
* Получаем правдоподобие: $$p(w|d) = \sum_{t \in T}p(w|t)p(t|d) $$

Полученная модель - probabilistic latent semantic analysis, pLSA, Вероятностный латентно-семантический анализ

http://www.machinelearning.ru/wiki/index.php?title=%D0%92%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%BD%D1%8B%D0%B9_%D0%BB%D0%B0%D1%82%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%B5%D0%BC%D0%B0%D0%BD%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7

Обучение:  
Нам нужны величины:
* $p(w|t)$ - вероятности слов в темах, обозначим $\phi_{wt}$

* $p(t|d)$ - вероятности тем в документах, обозначим $\theta_{td}$

E-шаг:
* фиксируем $\phi_{wt}$ и $\theta_{td}$
* вычисляем $$p(t|d,w) = \frac{\phi_{wt} \theta_{td}}{\sum_{s \in T}\phi_{ws} \theta_{sd}}$$ для всех тем, для каждого документа, для каждого термина
* вычисляем количество терминов, которое генерируется в документе $d$ темой $t$ $$n_{dwt} = n_{dw}p(t|d,w)$$

М-шаг:
* по вычисленным $p(t|d,w)$ обновить приближения модели $\phi_{wt}$ и $\theta_{td}$
* $$n_{wt} = \sum_d n_{dwt}$$ $$n_{td} = \sum_{w \in d} n_{dwt}$$ $$n_t=\sum_w n_{wt}$$
* $$\theta_{td} = \frac{n_{td}}{n_d}$$ $$\phi_{wt} = \frac{n_{wt}}{n_t}$$


Можно не хранить матрицу $n_{dwt}$, а итерироваться по документам и суммировать $n_{wt}$ и $n_{td}$
* Много локальных экстремумов
* Много параметров, модель переобучается
* Нужно достичь не локальный минимум, а добиться интерпретируемости - найти "хороший" минимум

## LDA

В общем случае, чтобы улучшить pLSA, в логарифм правдоподобия добавляют регуляризацию:

$$\sum_{d \in D} \sum_{w \in d} n_{dw} ln \sum_{t \in T} \phi_{wt} \theta_{td} + \sum_i \tau_i R_i(\Phi, \Theta)$$

Если добавить априорное распределение - распределение дирехле, получим алгоритм LDA - Latent Dirichlet Allocation

В итоге получаем "хорошее" интерпретируемое решение (лучше, чем с pLSA)


Один документ может содержать несколько тем.  
Составляем иерархическую модель:  
* первый уровень - смесь, компоненты которой отвечают за темы
* второй уровень - мультиномиальная переменная с априорным распределением Дирихле, которая определяет "распределение над темами" в документе

Обучение:
* сэмплирование по Гибсу
* online variational bayes

In [6]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

In [7]:
# import nltk
# nltk.download('wordnet')

In [8]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]        

In [9]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [10]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word=dictionary, passes=50)

In [12]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.076*"sugar" + 0.075*"good" + 0.075*"health"'), (1, u'0.084*"sister" + 0.084*"father" + 0.059*"sugar"'), (2, u'0.065*"driving" + 0.065*"pressure" + 0.064*"doctor"')]


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [15]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      62.201374        1       1  0.079634 -0.029123
2      22.339130        1       2 -0.073702 -0.036771
0      15.459492        1       3 -0.005933  0.065894, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
0     Default  0.000000     expert  0.000000  30.0000  30.0000
5     Default  0.000000       good  0.000000  29.0000  29.0000
28    Default  0.000000  lifestyle  0.000000  28.0000  28.0000
17    Default  0.000000     health  0.000000  27.0000  27.0000
4     Default  0.000000        say  0.000000  26.0000  26.0000
13    Default  2.000000      sugar  2.000000  25.0000  25.0000
11    Default  0.000000    suggest  0.000000  24.0000  24.0000
26    Default  0.000000      blood  0.000000  23.0000  23.0000
24    Default  0.000000     stress  0.000000  22.0000  22.0000
22    Default  0.000000        may  0.000000  21.0000  21.0000
19    Default  0.000000      cause  0.000000  20.0000  20.0000
9     Default  0.000000     doctor  0.000000  19.0000  19.0000
7     Default  0.000000  increased  0.000000  18.0000  18.0000
8     Default  1.000000    driving  1.000000  17.0000  17.0000
25    Default  1.000000   pressure  1.000000  16.0000  16.0000
12    Default  2.000000     father  2.000000  15.0000  15.0000
27    Default  2.000000     sister  2.000000  14.0000  14.0000
29    Default  1.000000       like  1.000000  13.0000  13.0000
33    Default  1.000000        bad  1.000000  12.0000  12.0000
1     Default  1.000000    consume  1.000000  11.0000  11.0000
23    Default  1.000000   practice  1.000000  10.0000  10.0000
2     Default  1.000000      dance  1.000000   9.0000   9.0000
6     Default  1.000000     spends  1.000000   8.0000   8.0000
34    Default  1.000000       time  1.000000   7.0000   7.0000
21    Default  1.000000     around  1.000000   6.0000   6.0000
18    Default  1.000000        lot  1.000000   5.0000   5.0000
16    Default  1.000000     school  1.000000   4.0000   4.0000
20    Default  1.000000      never  1.000000   3.0000   3.0000
15    Default  1.000000       feel  1.000000   2.0000   2.0000
30    Default  1.000000  sometimes  1.000000   1.0000   1.0000
...       ...       ...        ...       ...      ...      ...
13     Topic3  0.504510      sugar  2.228024   0.3817  -2.5784
26     Topic3  0.125385      blood  0.969339  -0.1783  -3.9706
24     Topic3  0.125385     stress  0.969339  -0.1783  -3.9706
22     Topic3  0.125385        may  0.969339  -0.1783  -3.9706
7      Topic3  0.125385  increased  0.969339  -0.1783  -3.9706
9      Topic3  0.125385     doctor  0.969339  -0.1783  -3.9706
19     Topic3  0.125385      cause  0.969339  -0.1783  -3.9706
11     Topic3  0.125385    suggest  0.969339  -0.1783  -3.9706
1      Topic3  0.125606    consume  1.179685  -0.3729  -3.9689
33     Topic3  0.125606        bad  1.179685  -0.3729  -3.9689
29     Topic3  0.125606       like  1.179686  -0.3729  -3.9689
2      Topic3  0.125494      dance  1.179793  -0.3739  -3.9697
6      Topic3  0.125494     spends  1.179793  -0.3739  -3.9697
34     Topic3  0.125494       time  1.179793  -0.3739  -3.9697
18     Topic3  0.125494        lot  1.179793  -0.3739  -3.9697
21     Topic3  0.125494     around  1.179793  -0.3739  -3.9697
23     Topic3  0.125494   practice  1.179793  -0.3739  -3.9697
15     Topic3  0.125438       feel  1.179870  -0.3744  -3.9702
30     Topic3  0.125438  sometimes  1.179870  -0.3744  -3.9702
16     Topic3  0.125438     school  1.179870  -0.3744  -3.9702
10     Topic3  0.125438    perform  1.179870  -0.3744  -3.9702
20     Topic3  0.125438      never  1.179870  -0.3744  -3.9702
31     Topic3  0.125438       well  1.179870  -0.3744  -3.9702
32     Topic3  0.125438      drive  1.179870  -0.3744  -3.9702
3      Topic3  0.125438      seems  1.179870  -0.3744  -3.9702
8      Topic3  0.125606    drivin

# https://www.kaggle.com/benhamner/nips-papers

In [16]:
import pandas as pd

ds = pd.read_csv('papers.csv')
ds.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [19]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string


stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in ds['paper_text']]        

In [23]:
doc_clean[0]

[u'767',
 u'selforganization',
 u'associative',
 u'database',
 u'application',
 u'hisashi',
 u'suzuki',
 u'suguru',
 u'arimoto',
 u'osaka',
 u'university',
 u'toyonaka',
 u'osaka',
 u'560',
 u'japan',
 u'abstract',
 u'efficient',
 u'method',
 u'selforganizing',
 u'associative',
 u'database',
 u'proposed',
 u'together',
 u'application',
 u'robot',
 u'eyesight',
 u'system',
 u'proposed',
 u'database',
 u'associate',
 u'input',
 u'output',
 u'first',
 u'half',
 u'part',
 u'discussion',
 u'algorithm',
 u'selforganization',
 u'proposed',
 u'aspect',
 u'hardware',
 u'produce',
 u'new',
 u'style',
 u'neural',
 u'network',
 u'latter',
 u'half',
 u'part',
 u'applicability',
 u'handwritten',
 u'letter',
 u'recognition',
 u'autonomous',
 u'mobile',
 u'robot',
 u'system',
 u'demonstrated',
 u'introduction',
 u'let',
 u'mapping',
 u'f',
 u'x',
 u'given',
 u'here',
 u'x',
 u'finite',
 u'infinite',
 u'set',
 u'another',
 u'finite',
 u'infinite',
 u'set',
 u'learning',
 u'machine',
 u'observes',
 u'se

In [25]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [29]:
doc_term_matrix[0]

[(0, 5),
 (1, 22),
 (2, 1),
 (3, 3),
 (4, 3),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 4),
 (9, 1),
 (10, 2),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 2),
 (15, 1),
 (16, 4),
 (17, 9),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 11),
 (22, 2),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 4),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 12),
 (34, 2),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 4),
 (46, 2),
 (47, 3),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 7),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 7),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 3),
 (74, 2),
 (75, 1),
 (76, 2),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 2),
 (81, 1),
 (82, 6),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 2),
 (87, 1),
 (88, 6),
 (89, 1),
 (90, 1),
 (91, 3),
 (92, 1),
 (93, 2),
 (94, 2),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 4),
 (100, 

In [32]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word=dictionary, passes=1)

In [33]:
for t in ldamodel.print_topics(num_topics=30, num_words=10):
    print(t)

(0, u'0.021*"network" + 0.009*"neural" + 0.008*"1" + 0.007*"input" + 0.007*"model" + 0.005*"output" + 0.005*"unit" + 0.005*"time" + 0.005*"2" + 0.004*"learning"')
(1, u'0.016*"1" + 0.012*"2" + 0.008*"model" + 0.008*"x" + 0.006*"0" + 0.006*"p" + 0.006*"algorithm" + 0.006*"graph" + 0.005*"r" + 0.005*"n"')
(2, u'0.013*"image" + 0.013*"model" + 0.010*"1" + 0.006*"2" + 0.005*"x" + 0.005*"3" + 0.005*"feature" + 0.004*"a" + 0.004*"using" + 0.004*"object"')
(3, u'0.014*"1" + 0.012*"model" + 0.008*"data" + 0.008*"x" + 0.007*"learning" + 0.005*"distribution" + 0.005*"2" + 0.005*"k" + 0.005*"0" + 0.005*"3"')
(4, u'0.022*"1" + 0.015*"2" + 0.015*"x" + 0.010*"algorithm" + 0.010*"k" + 0.009*"n" + 0.009*"0" + 0.007*"f" + 0.007*"function" + 0.006*"p"')
(5, u'0.016*"1" + 0.011*"0" + 0.009*"2" + 0.009*"x" + 0.009*"n" + 0.007*"p" + 0.007*"model" + 0.007*"j" + 0.006*"c" + 0.006*"r"')
(6, u'0.012*"model" + 0.011*"1" + 0.008*"0" + 0.008*"2" + 0.007*"data" + 0.005*"x" + 0.005*"k" + 0.005*"p" + 0.005*"b" + 0.0

### bigrams



In [41]:
doc_bigrams = [[t1 + '_' + t2 for t1, t2 in zip(doc, doc[1:])] for doc in doc_clean]

dictionary = corpora.Dictionary(doc_bigrams)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_bigrams]

In [43]:
doc_term_matrix[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 2),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 4),
 (25, 3),
 (26, 6),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 2),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 2),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),

In [44]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word=dictionary, passes=1)

In [45]:
for t in ldamodel.print_topics(num_topics=30, num_words=10):
    print(t)

(0, u'0.001*"reinforcement_learning" + 0.001*"et_al" + 0.001*"s_a" + 0.001*"1_2" + 0.000*"machine_learning" + 0.000*"value_function" + 0.000*"reward_function" + 0.000*"0_1" + 0.000*"0_0" + 0.000*"neural_network"')
(1, u'0.001*"1_1" + 0.001*"et_al" + 0.001*"0_1" + 0.000*"1_2" + 0.000*"machine_learning" + 0.000*"x_x" + 0.000*"figure_1" + 0.000*"2_2" + 0.000*"k_k" + 0.000*"figure_2"')
(2, u'0.001*"1_1" + 0.001*"1_2" + 0.001*"machine_learning" + 0.001*"et_al" + 0.001*"0_0" + 0.001*"x_x" + 0.001*"neural_information" + 0.001*"2_2" + 0.001*"0_1" + 0.001*"information_processing"')
(3, u'0.001*"et_al" + 0.001*"machine_learning" + 0.001*"1_2" + 0.001*"x_x" + 0.001*"1_1" + 0.001*"0_1" + 0.001*"processing_system" + 0.001*"0_0" + 0.001*"neural_network" + 0.000*"figure_1"')
(4, u'0.001*"et_al" + 0.001*"x_x" + 0.001*"1_2" + 0.001*"0_0" + 0.001*"information_processing" + 0.001*"processing_system" + 0.001*"0_1" + 0.000*"machine_learning" + 0.000*"neural_network" + 0.000*"gaussian_process"')
(5, u'0.001

In [47]:
#save model
# ldamodel.save('nips.bigrams')

#Load model
ldamodel = Lda.load('nips.bigrams')

In [49]:
for t in ldamodel.print_topics(num_topics=10, num_words=5):
    print(t)

(0, u'0.001*"reinforcement_learning" + 0.001*"et_al" + 0.001*"s_a" + 0.001*"1_2" + 0.000*"machine_learning"')
(1, u'0.001*"1_1" + 0.001*"et_al" + 0.001*"0_1" + 0.000*"1_2" + 0.000*"machine_learning"')
(2, u'0.001*"1_1" + 0.001*"1_2" + 0.001*"machine_learning" + 0.001*"et_al" + 0.001*"0_0"')
(3, u'0.001*"et_al" + 0.001*"machine_learning" + 0.001*"1_2" + 0.001*"x_x" + 0.001*"1_1"')
(4, u'0.001*"et_al" + 0.001*"x_x" + 0.001*"1_2" + 0.001*"0_0" + 0.001*"information_processing"')
(5, u'0.001*"et_al" + 0.001*"neural_network" + 0.000*"figure_2" + 0.000*"computer_vision" + 0.000*"arxiv_preprint"')
(6, u'0.003*"neural_network" + 0.001*"et_al" + 0.001*"0_0" + 0.001*"hidden_unit" + 0.001*"1_1"')
(7, u'0.002*"neural_network" + 0.001*"et_al" + 0.001*"neural_information" + 0.001*"figure_1" + 0.000*"1_2"')
(8, u'0.001*"neural_network" + 0.001*"et_al" + 0.001*"machine_learning" + 0.001*"1_1" + 0.001*"0_1"')
(9, u'0.001*"f_x" + 0.001*"1_2" + 0.001*"1_1" + 0.001*"0_1" + 0.001*"2_2"')


# https://www.kaggle.com/mrisdal/fake-news/data

In [49]:
ds = pd.read_csv('fake.csv', usecols = ['text'])
ds.dropna(axis=0, inplace=True, subset=['text'])
ds = ds.sample(frac=1.0)
ds['text'].head()

11512    Here's something interesting from The Unz Revi...
4868      Doomsday Election\nBy Mike Whitney November 0...
3093     \nA new Wikileaks email just exposed that Hill...
9415     \n“Go ahead. Look at them,” said Hillary Clint...
12030    « L’art de la guerre »\nComment voter « Non » ...
Name: text, dtype: object

In [50]:
len(ds)

12953

In [51]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string


stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word.decode('utf-8')) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in ds['text']]        

In [52]:
doc_clean[0]

[u'here',
 u'something',
 u'interesting',
 u'unz',
 u'review',
 u'recipient',
 u'name',
 u'pat',
 u'buchanan',
 u'read',
 u'book',
 u'wrote',
 u'them',
 u'credit',
 u'vdarecom',
 u'donald',
 u'trump',
 u'hadn\u2019t',
 u'crazy',
 u'thinskinned',
 u'moral',
 u'lout',
 u'hold',
 u'constitution',
 u'contempt',
 u'could',
 u'supported',
 u'him',
 u'maybe',
 u'mushy',
 u'moderate',
 u'think',
 u'alike',
 u'rod',
 u'dreher',
 u'tragedy',
 u'trump',
 u'american',
 u'conservative',
 u'october',
 u'24',
 u'2016',
 u'praising',
 u'new',
 u'york',
 u'time',
 u'token',
 u'conservative',
 u'ross',
 u'douthat',
 u'saying',
 u'danger',
 u'hillary',
 u'clinton',
 u'presidency',
 u'familiar',
 u'trump\u2019s',
 u'authoritarian',
 u'unknown',
 u'live',
 u'politics',
 u'already',
 u'they\u2019re',
 u'danger',
 u'elite',
 u'groupthink',
 u'beltway',
 u'power',
 u'worship',
 u'cult',
 u'presidential',
 u'action',
 u'service',
 u'dubious',
 u'ideal',
 u'they\u2019re',
 u'danger',
 u'recklessness',
 u'radica

In [53]:
from nltk import FreqDist

# use nltk fdist to get a frequency distribution of all words
fdist = FreqDist(word for d in doc_clean for word in d)


In [54]:
len(fdist)

213611

In [55]:
k = 50000
top_k_words = fdist.most_common(k)
top_k_words[-10:]

[(u'\u043f\u043e\u0435\u0437\u0434\u043e\u0432', 4),
 (u'seward', 4),
 (u'mcguire', 4),
 (u'guilty\u2019', 4),
 (u'posttrump', 4),
 (u'theistic', 4),
 (u'\u201dask', 4),
 (u'\u0448\u043a\u043e\u043b\u0430\u0445', 4),
 (u'approximated', 4),
 (u'bugging', 4)]

In [56]:
k = 15000
top_k_words = fdist.most_common(k)
top_k_words[-10:]

[(u'\u0432\u0430\u043c', 27),
 (u'ersten', 27),
 (u'risked', 27),
 (u'law\u2019s', 27),
 (u'lehman', 27),
 (u'transpired', 27),
 (u'22nd', 27),
 (u'scoundrel', 27),
 (u'\u042f\u043f\u043e\u043d\u0438\u0438', 27),
 (u'don\xe2\u20ac\u2122t', 27)]

In [57]:
top_k_words = dict(top_k_words)

In [58]:
doc_clean_freqs = [[w for w in doc if w in top_k_words] for doc in doc_clean]

In [59]:
doc_clean_freqs[0]

[u'here',
 u'something',
 u'interesting',
 u'unz',
 u'review',
 u'recipient',
 u'name',
 u'pat',
 u'buchanan',
 u'read',
 u'book',
 u'wrote',
 u'them',
 u'credit',
 u'vdarecom',
 u'donald',
 u'trump',
 u'hadn\u2019t',
 u'crazy',
 u'moral',
 u'hold',
 u'constitution',
 u'contempt',
 u'could',
 u'supported',
 u'him',
 u'maybe',
 u'moderate',
 u'think',
 u'alike',
 u'rod',
 u'tragedy',
 u'trump',
 u'american',
 u'conservative',
 u'october',
 u'24',
 u'2016',
 u'praising',
 u'new',
 u'york',
 u'time',
 u'token',
 u'conservative',
 u'ross',
 u'saying',
 u'danger',
 u'hillary',
 u'clinton',
 u'presidency',
 u'familiar',
 u'trump\u2019s',
 u'authoritarian',
 u'unknown',
 u'live',
 u'politics',
 u'already',
 u'they\u2019re',
 u'danger',
 u'elite',
 u'beltway',
 u'power',
 u'worship',
 u'cult',
 u'presidential',
 u'action',
 u'service',
 u'dubious',
 u'ideal',
 u'they\u2019re',
 u'danger',
 u'radicalism',
 u'doesn\u2019t',
 u'recognize',
 u'either',
 u'it\u2019s',
 u'convinced',
 u'idea',
 u'ma

In [60]:
doc_bigrams = [[t1 + '_' + t2 for t1, t2 in zip(doc, doc[1:])] for doc in doc_clean_freqs]

dictionary = corpora.Dictionary(doc_bigrams)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_bigrams]

In [ ]:
doc_term_matrix[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 3),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 2),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 2),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 5),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 1),
 (100, 1),

In [ ]:
Lda = gensim.models.ldamodel.LdaModel


num_topics = 100
chunksize = 300


# low alpha means each document is only represented by a small number of topics, and vice versa
# low eta means each topic is only represented by a small number of words, and vice versa

ldamodel = Lda(
    doc_term_matrix, 
    num_topics=num_topics, 
    id2word=dictionary, 
    alpha=1e-2, 
    eta=0.5e-2, 
    chunksize=chunksize, 
    minimum_probability=0.0, 
    passes=2, 
)

In [ ]:
#save model
ldamodel.save('fake.bigrams')

#Load model
# ldamodel = Lda.load('fake.bigrams')

In [ ]:
for t in ldamodel.print_topics(num_topics=100, num_words=5):
    print(t)

In [ ]:
ldamodel.show_topic(topicid=4, topn=20)

# VK walls

In [ ]:
def get_res_arr(filename):
    res_arr = []
    data = json.load(open(filename))
    for id in data:
        res_arr.append(data[id])
    #print(len(res_arr))
    return res_arr

path='./users_posts/'
super_arr = []
for filename in os.listdir(path):
    #print(filename)
    super_arr.extend(get_res_arr(path + filename))

print(len(super_arr))

40328


In [ ]:
stop = set(stopwords.words('russian') + stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    #print(doc)
    doc = str(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in super_arr]   
print(doc_clean[0:1])

In [ ]:


# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
doc_term_matrix[0]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word=dictionary, passes=1)

In [ ]:
#ldamodel.save('vk.unigrams')

In [ ]:
for t in ldamodel.print_topics(num_topics=30, num_words=10):
    print(t)

### биграммы

In [ ]:
doc_bigrams = [[t1 + '_' + t2 for t1, t2 in zip(doc, doc[1:])] for doc in doc_clean]

dictionary = corpora.Dictionary(doc_bigrams)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_bigrams]

In [4]:
Lda = gensim.models.ldamodel.LdaModel

In [7]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word=dictionary, passes=1)

NameError: name 'doc_term_matrix' is not defined

In [5]:
#ldamodel.save('vk.bigrams')
ldamodel = Lda.load('vk.bigrams')

In [6]:
for t in ldamodel.print_topics(num_topics=30, num_words=10):
    print(t[1])

0.009*"с_днем" + 0.009*"днем_рождения" + 0.007*"картинки_открытки" + 0.007*"открытки_♥" + 0.007*"с_днём" + 0.006*"днём_рождения" + 0.005*"новая_открытка" + 0.005*"открытки_поздравления" + 0.005*"😉_открытки" + 0.005*"поздравления_😉"
0.008*"ᅠ_ᅠ" + 0.006*"n╬═╬_n╬═╬" + 0.003*"спасибо_помощь" + 0.003*"от_души" + 0.003*"души_спасибо" + 0.003*"помощь_бою" + 0.002*"бою_беспредельшиком" + 0.002*"беспредельшиком_от" + 0.001*"в_наличии" + 0.001*"светлых_сил"
0.007*"n_n" + 0.001*"n_●" + 0.001*"n_n1" + 0.001*"—_это" + 0.001*"n_n2" + 0.001*"новогодние_подарки" + 0.001*"своим_друзьям" + 0.001*"n_n3" + 0.001*"игрушек_пожалуйста" + 0.001*"–_это"
0.003*"одержал_победу" + 0.003*"я_одержал" + 0.002*"победу_сражении" + 0.001*"победу_боссом" + 0.001*"13_13" + 0.001*"ответом_askfm" + 0.001*"поделилсялась_ответом" + 0.001*"я_получил" + 0.001*"askfm_поделилсялась" + 0.001*"принял_участие"
0.001*"новый_год" + 0.001*"—_это" + 0.001*"люблю_люблю" + 0.001*"друг_друга" + 0.001*"каждый_день" + 0.001*"социальной_сети

In [7]:
ldamodel.get_document_topics(doc_term_matrix[0])

NameError: name 'doc_term_matrix' is not defined

In [ ]:

# use nltk fdist to get a frequency distribution of all words
fdist = FreqDist(word for d in doc_clean for word in d)
len(fdist)

In [ ]:
k = 50000
top_k_words = fdist.most_common(k)
top_k_words[-10:]

In [ ]:
top_k_words = dict(top_k_words)

In [ ]:
doc_clean_freqs = [[w for w in doc if w in top_k_words] for doc in doc_clean]

In [ ]:
doc_clean_freqs[0]

In [ ]:
doc_bigrams = [[t1 + '_' + t2 for t1, t2 in zip(doc, doc[1:])] for doc in doc_clean_freqs]

dictionary = corpora.Dictionary(doc_bigrams)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_bigrams]

In [ ]:
doc_term_matrix[0]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel


num_topics = 100
chunksize = 300


# low alpha means each document is only represented by a small number of topics, and vice versa
# low eta means each topic is only represented by a small number of words, and vice versa

ldamodel = Lda(
    doc_term_matrix, 
    num_topics=num_topics, 
    id2word=dictionary, 
    alpha=1e-2, 
    eta=0.5e-2, 
    chunksize=chunksize, 
    minimum_probability=0.0, 
    passes=2, 
)

In [ ]:
#save model
ldamodel.save('fake.bigrams')

#Load model
# ldamodel = Lda.load('fake.bigrams')

In [ ]:
for t in ldamodel.print_topics(num_topics=100, num_words=5):
    print(t[1])

In [ ]:
ldamodel.show_topic(topicid=4, topn=20)

http://www.machinelearning.ru/wiki/images/8/82/BMMO11_14.pdf
http://www.machinelearning.ru/wiki/images/f/f7/DirichletProcessNotes.pdf 
